In [11]:
#Use findspark to confirm pyspark works
import findspark
import streamlit
findspark.init()

In [12]:
#Import pyspark
import pyspark
from pyspark import SparkContext, SparkConf
from collections import Counter
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import split, col

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

conf = SparkConf().setAppName('Project').setMaster('local')
sparkContext = SparkContext.getOrCreate(conf=conf) #SparkContext(conf = conf)

In [21]:
# Read in and assign column names
# Income data
Income = spark.read.csv('income_csv.csv',header=False)
Income = Income.toDF('Area_code','Area','Self_num_ind','Self_mean','Self_median','Num_ind','Mean','Median','Pens_num_ind','Pens_mean',
                     'Pens_median','Other_num_ind','Other_mean','Other_median','Tot_num_ind','Tot_mean','Tot_median','Tax_num_ind','Tax_mean','Tax_median','Tot_tax')

# Drop not applicable
Income = Income.na.drop()

# Remove original formatting
Income = Income.filter(Income.Area!='Region/County')

# Population data
Population = spark.read.csv('population_data.csv',header=False)
Population = Population.toDF('Code','Name','Geography','All ages','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15',
                     '16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37',
                     '38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59',
                     '60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80','81','82','83','84','85','86','87','88','89','90+')

# Drop not applicable
Population = Population.na.drop()

# Remove original formatting
Population = Population.filter(Population.Code!='Code')

# Income data
Statements = spark.read.csv('Transposed_Consolidated_Income_Statements.csv',header=False)
Statements = Statements.toDF('Company','Fina_period','State','Revenue','Cost','Gross','Other','Disp_close','Admin','Operating','Fin_out','Fin_in','Share_loss','Before_tax','Tax','Profit')

# Remove original formatting
Statements = Statements.drop(col('Company'))
Statements = Statements.drop(col('Fina_period'))
Statements = Statements.drop(col('State'))

# Remove original formatting
Statements = Statements.filter(Statements.Cost!='Cost of sales')

Population.show()

+---------+--------------------+--------------------+------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|     Code|                Name|           Geography|    All ages|      0|      1|      2|      3|      4|      5|      6|      7|      8|      9|     10|     11|     12|     13|     14|     15|     16|   

In [46]:
import numpy as np

# Condensing the data to only what is relevant
# Dropping self employement data

# Income condensed
Cond_Inc = Income

Cond_Inc = Cond_Inc.drop(col('Self_num_ind'))
Cond_Inc = Cond_Inc.drop(col('Self_mean'))
Cond_Inc = Cond_Inc.drop(col('Self_median'))

# Dropping pension data
Cond_Inc = Cond_Inc.drop(col('Pens_num_ind'))
Cond_Inc = Cond_Inc.drop(col('Pens_mean'))
Cond_Inc = Cond_Inc.drop(col('Pens_median'))

# Remove the non-applicable areas
Cond_Inc = Cond_Inc.filter(Cond_Inc.Area_code != 'Not applicable')

# Get all the area codes in the condensed income dataframe
codes_inc = [row.Area_code for row in Cond_Inc.select('Area_code').collect()]

# Population condensed
Cond_Pop = Population

# Dropping columns for children
for colu in range(0,16):
    Cond_Pop = Cond_Pop.drop('',col(str(colu)))

# Dropping repeated data in both population and income
Cond_Pop = Cond_Pop.drop(col('Geography'))
Cond_Pop = Cond_Pop.drop(col('Name'))

# Get codes for population table
codes_pop = [row.Code for row in Cond_Pop.select('Code').collect()]

for code in codes_pop[:]:
    if (code in codes_inc):
        codes_pop.remove(code)
        codes_inc.remove(code)


for i in range(0,len(codes_pop)):
    Cond_Pop = Cond_Pop.where(Cond_Pop.Code == codes_pop[i])

Cond_Pop.show()


Py4JJavaError: An error occurred while calling o6112.showString.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.AtLeastNNonNulls.copy(nullExpressions.scala:411)
	at org.apache.spark.sql.catalyst.expressions.AtLeastNNonNulls.withNewChildrenInternal(nullExpressions.scala:499)
	at org.apache.spark.sql.catalyst.expressions.AtLeastNNonNulls.withNewChildrenInternal(nullExpressions.scala:411)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$withNewChildren$2(TreeNode.scala:339)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$Lambda$1425/0x0000000840b2ec40.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.withNewChildren(TreeNode.scala:338)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:699)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:466)
	at org.apache.spark.sql